In [1]:
import pandas as pd
import numpy as np

# seller_name = ["seller_id", "breed", "good_id", "good_stock",
#                "depot_id", "brand", "place", "year", "level", "category"]
# buyer_name = ["buyer_id", "hold_time", "buy_count", "breed", "expect1", "value1",
#               "expect2", "value2", "expect3", "value3", "expect4", "value4", "expect5", "value5"]

df_seller = pd.read_csv("./seller.txt", header=0, low_memory=False, dtype=np.str)
df_buyer = pd.read_csv("./buyer.txt", header=0, low_memory=False, dtype=np.str)
df_seller.fillna(value='null', inplace=True)
df_buyer.fillna(value='null', inplace=True)
df_seller[['货物数量（张）']] = df_seller[['货物数量（张）']].values.astype(int)
df_buyer[['平均持仓时间', '购买货物数量']] = df_buyer[['平均持仓时间', '购买货物数量']].values.astype(int)

In [2]:
df_seller.head()

,卖方客户,品种,货物编号,货物数量（张）,仓库,品牌,产地,年度,等级,类别
0,999900000001,CF,CF2020011104087,8,314,null,新疆,2020,1230A,S1U2P3
1,999900000001,CF,CF2020011103822,1,335,null,新疆,2020,1228C,S2U3P2
2,999900000002,CF,CF2020011108737,2,309,null,非新疆,2020,2228A,S2U3P2
3,999900000002,CF,CF2020011101610,6,325,null,新疆,2020,4128C,S3U2P3
4,999900000002,CF,CF2020011104593,2,332,null,新疆,2020,4128C,S1U3P2


In [3]:
df_buyer.head()

,买方客户,平均持仓时间,购买货物数量,品种,第一意向,值,第二意向,值.1,第三意向,值.2,第四意向,值.3,第五意向,值.4
0,999800000000,3390,40,CF,仓库,314,产地,新疆,等级,4131B,类别,S3U3P3,null,null
1,999800000001,4952,5,CF,仓库,325,年度,2020,产地,新疆,类别,S3U4P2,等级,1130A
2,999800000002,3711,24,CF,仓库,301,产地,新疆,等级,4128A,类别,S3U3P3,年度,1920
3,999800000003,4100,56,CF,仓库,304,年度,1920,产地,新疆,类别,S3U3P2,null,null
4,999800000004,237,57,CF,仓库,301,类别,S2U3P2,产地,新疆,等级,2128C,年度,1920


In [38]:
tmp = df_buyer.groupby(["品种", "第一意向", "值", "第二意向", "值.1",
                        "第三意向", "值.2", "第四意向", "值.3", "第五意向", "值.4"]).count()
tmp.head(10)

买方客户  平均持仓时间  购买货物数量
品种 第一意向 值    第二意向 值.1  第三意向 值.2  第四意向 值.3   第五意向 值.4                         
CF null null null null null null null null  null null     849     849     849
   产地   新疆   null null null null null null  null null    1208    1208    1208
             仓库   301  null null null null  null null     119     119     119
                       年度   1920 null null  null null      24      24      24
                                 等级   1127B 类别   S4U2P2     1       1       1
                                      1128A 类别   S3U2P2     1       1       1
                                      1128B 类别   S4U3P1     1       1       1
                                      1128C 类别   S1U3P2     1       1       1
                                                 S2U3P1     1       1       1
                                                 S2U3P2     1       1       1

In [61]:
SCORE = {"CF": [33, 27, 20, 13, 7], "SR": [40, 30, 20, 10, 0]}

result = []


def get_score(buyer, seller):
    satisfy_score, breed = 0, buyer["品种"]
    satisfy_score += (SCORE[breed][0] if buyer["第一意向"] !=
                      "null" and seller[buyer["第一意向"]] == buyer["值"] else 0)
    satisfy_score += (SCORE[breed][1] if buyer["第二意向"] !=
                      "null" and seller[buyer["第二意向"]] == buyer["值.1"] else 0)
    satisfy_score += (SCORE[breed][2] if buyer["第三意向"] !=
                      "null" and seller[buyer["第三意向"]] == buyer["值.2"] else 0)
    satisfy_score += (SCORE[breed][3] if buyer["第四意向"] !=
                      "null" and seller[buyer["第四意向"]] == buyer["值.3"] else 0)
    satisfy_score += (SCORE[breed][4] if buyer["第五意向"] !=
                      "null" and seller[buyer["第五意向"]] == buyer["值.4"] else 0)
    val = min(buyer["购买货物数量"], seller["货物数量（张）"])
    score = val * satisfy_score
    return score




def assign(keys, buyers, sellers):
    tol_buy_count = buyers["购买货物数量"].sum()
    tol_seller_count = sellers["货物数量（张）"].sum()

    if tol_buy_count > tol_seller_count:
        print("*", keys, "(buyer:", len(buyers), ")(seller:", sellers.shape, ")")
        print("* 购买总数: {}, 出售总数: {}".format(tol_buy_count, tol_seller_count))
    else:
        buyers.sort_values(by="购买货物数量", inplace=True, ascending=False)
        first_buyer = buyers.iloc[0]
        breed = first_buyer["品种"]
        exp = [first_buyer["第一意向"], first_buyer["第二意向"],
               first_buyer["第三意向"], first_buyer["第四意向"], first_buyer["第五意向"]]
        exp_val = [first_buyer["值"], first_buyer["值.1"],
                   first_buyer["值.2"], first_buyer["值.3"], first_buyer["值.4"]]
        score_name = ["s1", "s2", "s3", "s4", "s5"]
        for i, it in enumerate(score_name):
            if exp[i] == "null":
                sellers[it] = 0
                continue
            sellers[it] = sellers[exp[i]].apply(
                lambda it: SCORE[breed][i] if it != "null" and it == exp_val[i] else 0)

        sellers["score"] = (sellers["s1"] + sellers["s2"] +
                            sellers["s3"] + sellers["s4"] + sellers["s5"]) * sellers["货物数量（张）"]
        sellers.sort_values(by="score", ascending=False)

        print("--------------------------------------")
        print(sellers["s1"].max(), sellers["s1"].min())
        print(sellers["s2"].max(), sellers["s2"].min())
        print(sellers["s3"].max(), sellers["s3"].min())
        print(sellers["s4"].max(), sellers["s4"].min())
        print(sellers["s5"].max(), sellers["s5"].min())
        print(sellers["score"].max(), sellers["score"].min())
    
        index = 0
        for idx, buyer in buyers.iterrows():
#             print("idx: ", idx)
            while buyer["购买货物数量"] > 0 and index < len(sellers.index):
                seller = sellers.iloc[index]
                val = min(buyer["购买货物数量"], seller["货物数量（张）"])
                seller["货物数量（张）"] -= val
                buyer["购买货物数量"] -= val

                expects = []
                for i, it in enumerate(score_name):
                    if seller[it] > 0:
                        expects.append(i + 1)
                if len(expects) == 0:
                    expects = [0]

                record = [seller["卖方客户"], buyer["买方客户"], buyer["品种"],
                          seller["货物编号"], seller["仓库"], val, expects]
                result.append(record)
                if seller["货物数量（张）"] <= 0:
#                     sellers.drop(sellers.index[0], inplace=True)
                    index += 1


# df_buyers = df_buyer.groupby(["品种", "第一意向", "值"], dropna=False)
df_buyers = df_buyer.groupby(["品种", "第一意向", "值", "第二意向", "值.1",
                              "第三意向", "值.2", "第四意向", "值.3", "第五意向", "值.4"], dropna=False)
print(len(df_buyers))
for i, it in enumerate(df_buyers):
#     keys, buyers = it[0], it[1]
#     print(keys, len(buyers))
    if i % 500 == 0:
        print(i)
    if keys[1] == 'null':
        continue
#     sellers = df_seller[(df_seller["品种"] == keys[0]) &
#                         (df_seller[keys[1]] == keys[2])]
#     assign(keys, buyers, sellers)
#     break

54106
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
20000
20500
21000
21500
22000
22500
23000
23500
24000
24500
25000
25500
26000
26500
27000
27500
28000
28500
29000
29500
30000
30500
31000
31500
32000
32500
33000
33500
34000
34500
35000
35500
36000
36500
37000
37500
38000
38500
39000
39500
40000
40500
41000
41500
42000
42500
43000
43500
44000
44500
45000
45500
46000
46500
47000
47500
48000
48500
49000
49500
50000
50500
51000
51500
52000
52500
53000
53500
54000
